### Auto Loader - Streaming Mode

In [0]:
%sql
select current_catalog();

In [0]:

dbutils.fs.ls('/Volumes/workspace/default/lateflow_connect_volume')

In [0]:
landing_zone = '/Volumes/workspace/default/lateflow_connect_volume'
orders_data = landing_zone + '/orders_data'
checkpoint_path = landing_zone + '/Streamingcheckpoint'

In [0]:
orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("cloudFiles.inferSchema", "true")
.option("cloudFiles.inferColumnTypes", "true")
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data))

In [0]:
orders_df.printSchema();

In [0]:
orders_df.printSchema();

In [0]:
%sql
drop table if exists ordersdeltastream;
create table ordersdeltastream;

#### Upload data into volume and run the below job
### for streaming mode the trigger(processingTime=<>)
## in serverless compute , processingTime is not supported

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(processingTime='10 seconds')
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from workspace.default.ordersdelta;

In [0]:
display(dbutils.fs.ls('/Volumes/workspace/default/lateflow_connect_volume/checkpoint/'))

#### upload another file with same schema

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdelta"))

In [0]:
%sql
select * from ordersdelta;

#### add another file with different schema

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdelta"))

In [0]:
%sql
select * from ordersdelta;

In [0]:
%sql
desc extended ordersdelta;

In [0]:
%sql
show databases;

In [0]:
%sql
select current_database();

In [0]:
%sql
show tables;

#### uplaod another file with mismatch data types

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdelta"))

In [0]:
%sql
select * from ordersdelta;

### How does AUTO LOADER Track the ingestion process?